In [29]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import os, codecs
%matplotlib inline
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn import metrics as skmetrics
import scipy.stats

# Model evaluation

In [30]:
model = gensim.models.Word2Vec.load('/home/bahbbc/workspace/masters-big5/models/tweet50-600.model')
model.init_sims(replace=True)

2017-08-05 18:29:55,261 : INFO : loading Word2Vec object from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model
2017-08-05 18:30:01,950 : INFO : loading wv recursively from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model.wv.* with mmap=None
2017-08-05 18:30:01,951 : INFO : loading syn0 from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model.wv.syn0.npy with mmap=None
2017-08-05 18:30:14,720 : INFO : setting ignored attribute syn0norm to None
2017-08-05 18:30:14,761 : INFO : loading syn1neg from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model.syn1neg.npy with mmap=None
2017-08-05 18:32:10,652 : INFO : setting ignored attribute cum_table to None
2017-08-05 18:32:11,033 : INFO : loaded /home/bahbbc/workspace/masters-big5/models/tweet50-600.model
2017-08-05 18:34:14,134 : INFO : precomputing L2-norms of word weight vectors


In [31]:
num_features= 600

## Dictionary features

In [32]:
personalities = pd.read_csv('~/personality-normalized-word2vec-norm.csv', encoding='utf-8')
personalities.shape

(1039, 186)

In [33]:
personalities.shape

(1039, 186)

In [34]:
personalities = personalities.convert_objects(convert_numeric=True)

/usr/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


In [36]:
personalities.extraversion_m.value_counts()

0    529
1    510
Name: extraversion_m, dtype: int64

In [37]:
train_data, test_data = train_test_split(personalities, test_size=0.3, random_state=42)

In [38]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    #print words.shape
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.wv.vocab:
            mean.append(wv.wv.syn0norm[wv.wv.vocab[word].index])
            all_words.add(wv.wv.vocab[word].index)
        #print mean

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(num_features,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list ])

In [39]:
def w2v_tokenize_text(text):
    tokens = []
    if text is np.nan:
        return []
    for sent in nltk.sent_tokenize(text, language='portuguese'):
        for word in nltk.word_tokenize(sent, language='portuguese'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [40]:
test_tokenized = test_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values
train_tokenized = train_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values

In [41]:
%%time
X_train_word_average = word_averaging_list(model,train_tokenized)
X_test_word_average = word_averaging_list(model,test_tokenized)

2017-08-05 18:39:41,248 : WARNING : cannot compute similarity with no input []
2017-08-05 18:39:43,348 : WARNING : cannot compute similarity with no input []
2017-08-05 18:39:43,358 : WARNING : cannot compute similarity with no input []
2017-08-05 18:39:44,469 : WARNING : cannot compute similarity with no input []
2017-08-05 18:39:44,854 : WARNING : cannot compute similarity with no input []
2017-08-05 18:39:45,435 : WARNING : cannot compute similarity with no input []
2017-08-05 18:39:46,509 : WARNING : cannot compute similarity with no input []
2017-08-05 18:39:46,627 : WARNING : cannot compute similarity with no input []
2017-08-05 18:39:46,953 : WARNING : cannot compute similarity with no input []
2017-08-05 18:39:47,195 : WARNING : cannot compute similarity with no input []
2017-08-05 18:39:47,282 : WARNING : cannot compute similarity with no input []


CPU times: user 5.85 s, sys: 408 ms, total: 6.25 s
Wall time: 11.2 s


In [42]:
del model

In [43]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.externals import joblib

force = False
model_trainer = RandomizedSearchCV(
    n_iter=1, 
    estimator=DecisionTreeRegressor(),
    param_distributions={
        "max_features": ["log2"],
        "random_state": [42],
        "criterion":['mse']
    },
    verbose=True,
    refit=True,
    cv=10,
    n_jobs=-1
)

Os resultados são ainda piores quando não se usa o ponto flutuante - ou seja, os inteiros

## Extraversion evaluation

In [47]:
train, test = X_train_word_average, X_test_word_average

In [49]:
%%time
model_trainer.fit(train, train_data['extraversion'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits
CPU times: user 201 ms, sys: 74.9 ms, total: 276 ms
Wall time: 396 ms


[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished


In [50]:
print train.shape
print train_data['extraversion'].shape

(727, 600)
(727,)


In [51]:
yp = model.predict(test)
yt =  test_data['extraversion']

In [52]:
skmetrics.mean_squared_error(yt, yp)

1.2838677795584046

In [53]:
skmetrics.mean_squared_error(yt, yp)**0.5

1.1330788937926628

In [54]:
skmetrics.r2_score(yt, yp)

-0.78645709383080131

In [55]:
scipy.stats.pearsonr(yt,yp)

(0.12730673074432702, 0.024524144537372863)

## Agreableness evaluation

In [56]:
%%time
model_trainer.fit(train, train_data['agreeableness'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished


CPU times: user 229 ms, sys: 89.8 ms, total: 319 ms
Wall time: 513 ms


In [57]:
yp = model.predict(test)
yt = test_data['agreeableness']

In [58]:
skmetrics.r2_score(yt, yp)

-0.92788668583170786

In [59]:
scipy.stats.pearsonr(yt,yp)

(-0.055005221108767287, 0.33283490452078279)

## Conscientiousness evaluation

In [60]:
%%time
model_trainer.fit(train, train_data['conscientiousness'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished


CPU times: user 242 ms, sys: 87.9 ms, total: 330 ms
Wall time: 514 ms


In [61]:
yp = model.predict(test)
yt = test_data['conscientiousness']

In [62]:
skmetrics.mean_squared_error(yt, yp)

0.87725200320512819

In [63]:
skmetrics.r2_score(yt, yp)

-1.2801062811163821

In [64]:
yp

array([ 2.67 ,  2.44 ,  3.22 ,  2.89 ,  1.89 ,  3.44 ,  4.   ,  3.78 ,
        2.22 ,  3.11 ,  2.33 ,  3.   ,  4.   ,  4.22 ,  4.22 ,  4.22 ,
        2.56 ,  2.22 ,  3.67 ,  3.11 ,  3.67 ,  3.33 ,  4.11 ,  2.   ,
        4.   ,  4.56 ,  2.56 ,  2.   ,  3.11 ,  3.78 ,  3.89 ,  3.78 ,
        3.11 ,  3.44 ,  3.78 ,  3.56 ,  2.78 ,  2.33 ,  2.67 ,  2.44 ,
        3.33 ,  1.89 ,  3.22 ,  2.   ,  1.89 ,  3.67 ,  4.33 ,  3.67 ,
        2.78 ,  2.89 ,  1.78 ,  2.22 ,  3.11 ,  4.   ,  4.56 ,  2.   ,
        3.33 ,  3.56 ,  4.67 ,  2.78 ,  4.11 ,  2.67 ,  3.44 ,  3.44 ,
        3.11 ,  3.56 ,  1.67 ,  2.33 ,  2.78 ,  3.67 ,  2.89 ,  3.33 ,
        4.67 ,  3.   ,  3.56 ,  3.67 ,  2.67 ,  3.67 ,  4.22 ,  4.78 ,
        2.67 ,  4.22 ,  3.78 ,  2.44 ,  4.11 ,  3.165,  2.56 ,  4.67 ,
        3.11 ,  2.78 ,  2.56 ,  2.89 ,  3.56 ,  4.11 ,  2.89 ,  3.44 ,
        4.11 ,  3.67 ,  3.56 ,  2.33 ,  3.56 ,  2.78 ,  2.44 ,  2.44 ,
        2.33 ,  3.56 ,  3.33 ,  2.56 ,  2.89 ,  3.56 ,  2.   ,  4.89 ,
      

In [65]:
yt

310     3.33
809     3.33
309     3.33
31      2.89
544     3.11
96      4.56
883     3.33
109     4.11
833     3.67
1034    2.67
917     3.56
286     3.44
868     3.67
277     3.67
981     3.33
817     3.11
136     4.33
923     3.56
996     3.89
210     2.89
244     3.11
289     3.11
101     3.11
911     4.11
59      1.89
1036    3.56
667     3.00
290     2.89
825     3.44
760     3.89
        ... 
370     2.67
611     3.78
352     4.11
77      2.67
84      3.67
436     3.56
82      2.89
1015    2.33
535     2.78
615     3.78
552     3.78
350     3.00
547     4.56
518     2.67
5       3.11
688     3.22
104     2.56
97      3.00
306     3.00
711     3.44
408     2.22
920     3.33
281     3.22
266     3.67
478     4.00
605     3.89
346     3.56
513     2.78
928     2.33
841     3.44
Name: conscientiousness, dtype: float64

In [66]:
scipy.stats.pearsonr(yt,yp)

(0.046552149818424413, 0.41254751834291314)

## Neuroticism evaluation


In [81]:
%%time
model_trainer.fit(train, train_data['neuroticism'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits
CPU times: user 228 ms, sys: 87.3 ms, total: 315 ms
Wall time: 442 ms


[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished


In [82]:
yp = model.predict(test)
yt = test_data['neuroticism']

In [83]:
skmetrics.mean_squared_error(yt, yp)

1.3451790776353276

In [84]:
skmetrics.r2_score(yp, yt)

-0.82336221470254678

In [85]:
scipy.stats.pearsonr(yt,yp)

(0.062948675910721591, 0.26763209406463695)

## Openess evaluation

In [72]:
%%time
model_trainer.fit(train, train_data['openness'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished


CPU times: user 224 ms, sys: 79.7 ms, total: 303 ms
Wall time: 521 ms


In [73]:
yp = model.predict(test)
yt = test_data['openness']

In [74]:
skmetrics.mean_squared_error(yt, yp)

0.73719551282051277

In [78]:
skmetrics.r2_score(yp, yt)

-0.77222162265885208

In [79]:
scipy.stats.pearsonr(yp, yt)

(0.079369961650818416, 0.16195305249470729)